# Function to fit our analytical model to the SF-gas from TNG

Our fit procedure is also described in section 3 of van son et al in prep.




In [1]:
import h5py as h5
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u

from scipy import interpolate
from scipy.optimize import minimize

# from astropy.cosmology import WMAP9, z_at_value
from astropy.cosmology import Planck18  as cosmo# Planck 2018
from astropy.cosmology import z_at_value

############################
# Custom scripts
import paths
import get_ZdepSFRD as Z_SFRD
import importlib
importlib.reload(Z_SFRD)

Cosmol_sim_location = '/Users/lieke/surfdrive/Documents/CompareCOMPAS/' + "SFRMetallicityFromGasTNG100.hdf5"


# First read the TNG data 
## And convert it to a SFDR in Msun/yr/Mpc^-3

***
We will have simulation data == TNG in our case
and model data  == our analytical function
***


In [2]:
##########################################
# Simulated SFRD data (from TNG)
##########################################
## Read in the pure simulation data
with h5.File(Cosmol_sim_location, "r") as f:
    MetalBins     = f["MetalBins"][:]
    Lookbacktimes = f["Lookbacktimes"][:]
    BoxSfr        = f["Sfr"][:]
# Convert SFR from sfr/box to sfr Mpc-3
littleh  = 0.6774
Rbox     = 75/littleh
Sim_SFRD = BoxSfr / Rbox**3 *u.Mpc**-3
Sim_SFRD = Sim_SFRD.value

## Adjust what metallicities to include 
# The minimum metallicity in COMPAS is 10^-4, so there is no
# use in fitting to the 10^-7 metallicity tail
Sim_center_Zbin = (MetalBins[:-1] + MetalBins[1:])/2.
low_bound_Z_ind  = np.where(Sim_center_Zbin > 1e-5)[0]# index of Sim_center_Zbin, where Z > 1e-5
tofit_Sim_metals = Sim_center_Zbin[low_bound_Z_ind]   

## Reverse the time axis of the SFRD and lookback time for the fit
tofit_Sim_SFRD      = Sim_SFRD[:,low_bound_Z_ind][::-1]
tofit_Sim_lookbackt = Lookbacktimes[::-1] 

## Convert lookback times to redshifts
# the last value of Lookbacktimes = 0, which is problematic for z calculation
redshifts_Sim = [z_at_value(cosmo.lookback_time,t*u.Gyr) for t in Lookbacktimes[:-1]] 
redshifts_Sim.append(0) # put redshift zero back at the end
redshifts_Sim = np.array(redshifts_Sim)


# Interpolate the TNG data

Using scipy interpolate
https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp2d.html

f = interpolate.interp2d(x, y, z)



In [3]:
#########################################
# Interpolate the simulation data
f_interp = interpolate.interp2d(tofit_Sim_lookbackt, tofit_Sim_metals, tofit_Sim_SFRD.T, kind='cubic')

#########################################
# Retrieve values at higher res regular intervals
redshift_interp_Sim         = np.arange(0, 10.1, 0.05)
Lookbacktimes_interp_Sim    = [cosmo.lookback_time(z).value for z in redshift_interp_Sim]

log_tofit_Sim_metals        =  np.log10(tofit_Sim_metals)
metal_interp_Sim            = np.logspace(min(log_tofit_Sim_metals), max(log_tofit_Sim_metals), 50)

SFRD_interp_Sim             = f_interp(Lookbacktimes_interp_Sim,metal_interp_Sim)

print('np.shape(Lookbacktimes_interp_Sim)', np.shape(Lookbacktimes_interp_Sim),'np.shape(metal_interp_Sim)',
      np.shape(metal_interp_Sim), 'np.shape(SFRD_interp_Sim)', np.shape(SFRD_interp_Sim) )


np.shape(Lookbacktimes_interp_Sim) (202,) np.shape(metal_interp_Sim) (50,) np.shape(SFRD_interp_Sim) (50, 202)


In [4]:

##################################################
# muz =-0.09, mu0 =0.026, sigma =1.9, alpha=-3.3
##################################################
def calc_chi_square(simulation_SFRD = [], simulation_metals = [], simulation_redshifts = [],  
                    mu_0_list = 0.026, muz_list =-0.09, sigma0_list = 1.9, sigmaz_list = 1.9, alpha_list =-3.3,
                    sf_a =0.01 , sf_b=2.6, sf_c=3.2 , sf_d=6.2):
    """
    Calculate the distribution of metallicities at different redshifts using a log skew normal distribution
    that is basically a skew normal distribution, but then with the random variable x = ln(Z)

    NOTE: This assumes that metallicities in COMPAS are drawn from a flat in log distribution

    Args:
        fit_metals              --> [float]          metals used for fitt
        Redshifts               --> [float]          redshihts used to fit
        simulation_SFRD           --> [float]        cosmological simulation SFRD to fit to
        
        mu_0_list    = -0.23    --> [float]          location (mean in normal) at redshift 0
        muz_list = 0.035    --> [float]          redshift evolution of the location
        sigma0_list  = 0.39     --> [float]          Scale at redshift 0 (variance in normal)
        sigmaz_list  = 0.0      --> [float]          redshift evolution of Scale (variance in normal)
        alpha_list   = 0.0      --> [float]          shape (skewness, alpha = 0 retrieves normal dist)

        sf_a                    --> [float]          SFR(z) parameter (shape of Madau & Dickenson 2014)
        sf_b                    --> [float]          SFR(z) parameter (shape of Madau & Dickenson 2014)
        sf_c                    --> [float]          SFR(z) parameter (shape of Madau & Dickenson 2014)
        sf_d                    --> [float]          SFR(z) parameter (shape of Madau & Dickenson 2014)

    Returns:
        tot_chi_square          --> [float ] 
    """
    
    """
    ######################################

    """  
    print('muz_list', muz_list, 'mu_0_list',mu_0_list, 'sigma0_list',sigma0_list,'sigmaz_list',sigmaz_list, 'alpha_list',alpha_list,
          'sf_a', sf_a, 'sf_b', sf_b, 'sf_c', sf_c, 'sf_d', sf_d)
        
    #####################################
    # Get the SFR
    # Madau & Fragos 2017: a=0.01, b=2.6, c=3.2,  d=6.2
    sfr = Z_SFRD.Madau_Dickinson2014(simulation_redshifts, a=sf_a, b=sf_b, c=sf_c, d=sf_d) # Msun year-1 Mpc-3 

    # Get dPdZ 
    dPdlogZ, redshifts, metallicities, step_logZ, p_draw_metallicity = \
                    Z_SFRD.skew_metallicity_distribution(mu_z = muz_list, mu_0 = mu_0_list,
                                                  sigma_0= sigma0_list, sigma_z=sigmaz_list, alpha = alpha_list, 
                                                  metals=simulation_metals, redsh = simulation_redshifts)
        
    log_fit_metallicities = np.log10(simulation_metals)
    step_fit_logZ         = np.diff(log_fit_metallicities)[0]
    
    ######################################
    # For each redshift in the TNG data:
    tot_chi_square = 0
        
    for redshift_i in range(len(redshifts)):
        #print(redshift_i, 'at redshift', redshifts[redshift_i])
        ######################################
        # Now the SFRD = sfr x dPdZ
        model_SFRD = sfr[redshift_i] *dPdlogZ[redshift_i,:] # Msun year-1 Mpc-3 

        # Model comes in dP/dlogZ, so should your sim-data 
        sim   = simulation_SFRD[redshift_i,:]/step_fit_logZ  # 
        model = model_SFRD.value 
        
        ###################
        # Zero values are troublesome in Chi_squared!
        # only fit where sim > x 
        common_bool = np.logical_and(model > 1e-11, sim > 1e-11)
        model       = model[common_bool]
        sim         = sim[common_bool]

        ###################
        # Actual CHI_squared
        chi_square = ((sim - model )**2)/np.sum(model) 
        
        tot_chi_square += np.sum(chi_square, axis = -1)


    ######################################
    # Minimum Chi_squared taking all redshift into aaccount
    print(tot_chi_square)
    
    return tot_chi_square


    
    

# Run your chi square calculations

## and leave the refinement up to scipy minimize

In [5]:
#################################################################
## Function wrapper to minimize the Chi_square
#################################################################
def test_chi(x0 = [-0.09, 0.026, 1.9, 0.1, -3.3,0.01, 2.6, 3.2, 6.2] ):
    chi_square_matix = calc_chi_square(simulation_SFRD = SFRD_interp_Sim.T, simulation_metals = metal_interp_Sim, simulation_redshifts = redshift_interp_Sim, 
                                       muz_list =x0[0], mu_0_list =x0[1],sigma0_list =x0[2], sigmaz_list=x0[3], alpha_list =x0[4],
                                       sf_a =x0[5], sf_b=x0[6], sf_c=x0[7], sf_d=x0[8])
    
    return chi_square_matix


# BEST GUESS
x0 = np.array([-0.15, 0.026, 1.1, 0.1, -3.3, 0.01, 2.6, 3.2, 6.2])
# FIT
res = minimize(test_chi, x0= x0, method ='nelder-mead', options = {'maxiter': 5000})




muz_list -0.15 mu_0_list 0.026 sigma0_list 1.1 sigmaz_list 0.1 alpha_list -3.3 sf_a 0.01 sf_b 2.6 sf_c 3.2 sf_d 6.2
21.070518043232298
muz_list -0.1575 mu_0_list 0.026 sigma0_list 1.1 sigmaz_list 0.1 alpha_list -3.3 sf_a 0.01 sf_b 2.6 sf_c 3.2 sf_d 6.2
21.188469645174486
muz_list -0.15 mu_0_list 0.0273 sigma0_list 1.1 sigmaz_list 0.1 alpha_list -3.3 sf_a 0.01 sf_b 2.6 sf_c 3.2 sf_d 6.2
20.981848330510953
muz_list -0.15 mu_0_list 0.026 sigma0_list 1.1550000000000002 sigmaz_list 0.1 alpha_list -3.3 sf_a 0.01 sf_b 2.6 sf_c 3.2 sf_d 6.2
21.292502263042792
muz_list -0.15 mu_0_list 0.026 sigma0_list 1.1 sigmaz_list 0.10500000000000001 alpha_list -3.3 sf_a 0.01 sf_b 2.6 sf_c 3.2 sf_d 6.2
21.176248381304024
muz_list -0.15 mu_0_list 0.026 sigma0_list 1.1 sigmaz_list 0.1 alpha_list -3.465 sf_a 0.01 sf_b 2.6 sf_c 3.2 sf_d 6.2
20.976936394453386
muz_list -0.15 mu_0_list 0.026 sigma0_list 1.1 sigmaz_list 0.1 alpha_list -3.3 sf_a 0.0105 sf_b 2.6 sf_c 3.2 sf_d 6.2
19.648225449562954
muz_list -0.15 mu

/Users/lieke/surfdrive/Documents/SFRD_fit/src/scripts/get_ZdepSFRD.py:107: RuntimeWarning: divide by zero encountered in true_divide
  mu_metallicities = np.log(mean_metallicities/(2.*PHI) * 1./(np.exp(0.5*sigma**2) )  )
/Users/lieke/surfdrive/Documents/SFRD_fit/src/scripts/get_ZdepSFRD.py:127: RuntimeWarning: invalid value encountered in true_divide
  dPdlogZ = dPdlogZ /norm[:,np.newaxis]



muz_list -0.14824532419089478 mu_0_list 0.026223790620428438 sigma0_list 1.0711035049568567 sigmaz_list 0.09801055787733509 alpha_list -3.3963054286992067 sf_a 0.009927637101557628 sf_b 2.7400745352938736 sf_c 3.3723994280539946 sf_d 5.926707724846225
10.32896441080387
muz_list -0.1453656842991594 mu_0_list 0.02584197180082543 sigma0_list 1.12858466792395 sigmaz_list 0.0959086516717498 alpha_list -3.362932972525366 sf_a 0.0102455756618489 sf_b 2.728047202944912 sf_c 3.357596557470658 sf_d 5.950173639009583
10.559381262489145
muz_list -0.15453359585352225 mu_0_list 0.02563249075915548 sigma0_list 1.093533829002006 sigmaz_list 0.09432883367714703 alpha_list -3.3547873047829455 sf_a 0.01023308508342727 sf_b 2.739065770525112 sf_c 3.3711578714155177 sf_d 5.9286758742901755
9.999513955997003
muz_list -0.16013887114653852 mu_0_list 0.02523296110443967 sigma0_list 1.0947520466580163 sigmaz_list 0.09066334653766844 alpha_list -3.3547941258986977 sf_a 0.01026663722733569 sf_b 2.787021152358313

2.8582106534606924
muz_list -0.14793404548295558 mu_0_list 0.02431464395667174 sigma0_list 1.0619557453108162 sigmaz_list 0.08636812565812617 alpha_list -3.477079286827726 sf_a 0.010665421066526334 sf_b 2.9083548211666868 sf_c 3.59710273676481 sf_d 5.761896072487282
3.990429808207018
muz_list -0.15378166969961996 mu_0_list 0.023150673966379397 sigma0_list 1.0388697849066064 sigmaz_list 0.0783128538255321 alpha_list -3.4959354966786007 sf_a 0.010989209223955868 sf_b 2.9522265779191486 sf_c 3.8324349243628437 sf_d 5.395202398669785
3.4008455030119884
muz_list -0.14901861333380326 mu_0_list 0.024016317213032084 sigma0_list 1.0247495058807723 sigmaz_list 0.07983664224548331 alpha_list -3.4676651554879587 sf_a 0.010619096654743929 sf_b 3.0053180097418806 sf_c 3.8241099079067613 sf_d 5.41121482690117
3.2268550536821823
muz_list -0.15009591284393375 mu_0_list 0.023907819929239515 sigma0_list 1.022697919764854 sigmaz_list 0.0750602156463355 alpha_list -3.535115573461583 sf_a 0.0108964839294470

1.8668964431808528
muz_list -0.1281719951291747 mu_0_list 0.023523954640205215 sigma0_list 0.972489661115119 sigmaz_list 0.07381520320115526 alpha_list -3.5262025791846945 sf_a 0.011327142256612577 sf_b 3.144135127967145 sf_c 3.8765397278209646 sf_d 5.915738241283256
2.7656891911349972
muz_list -0.11588011596104132 mu_0_list 0.027247040944132685 sigma0_list 0.9641283854999531 sigmaz_list 0.07449706661431066 alpha_list -3.3920542994286453 sf_a 0.01109186236226761 sf_b 3.1915738095794453 sf_c 3.5819407686133378 sf_d 6.752659081172999
1.6498400476751256
muz_list -0.10532587144892272 mu_0_list 0.02659495221751646 sigma0_list 0.9182920323489892 sigmaz_list 0.06708717662836658 alpha_list -3.5183390342426244 sf_a 0.011433640336050762 sf_b 3.308257659295935 sf_c 3.6969252857604755 sf_d 6.945135954972851
1.7338982778152112
muz_list -0.10004616159155971 mu_0_list 0.026805590387070626 sigma0_list 0.9258604183752561 sigmaz_list 0.07209073035029018 alpha_list -3.5065700664494637 sf_a 0.011815982037

...

0.137671220441297
muz_list -0.04674451752333875 mu_0_list 0.024740476136102564 sigma0_list 1.1157475483770547 sigmaz_list 0.04821636323481056 alpha_list -1.7057669579393326 sf_a 0.06566846950066979 sf_b 1.4836914633655107 sf_c 4.449460701903013 sf_d 5.905745276247524
0.13767138208171842
muz_list -0.04676266031707513 mu_0_list 0.024740253084534987 sigma0_list 1.1160690808862372 sigmaz_list 0.04822213416667121 alpha_list -1.7068075311182653 sf_a 0.06566414301443763 sf_b 1.4837794501523982 sf_c 4.449714699992816 sf_d 5.906064831065908
0.13767130898993896
muz_list -0.046741604235998546 mu_0_list 0.02473816560019275 sigma0_list 1.1162059179120192 sigmaz_list 0.048229643942429706 alpha_list -1.706624177951502 sf_a 0.06568176506566054 sf_b 1.483729873985971 sf_c 4.449212181020548 sf_d 5.904621799317251
0.13767134127610212
muz_list -0.04675116352515429 mu_0_list 0.024738373312188595 sigma0_list 1.1160592062466788 sigmaz_list 0.04821488010448295 alpha_list -1.7060166059064445 sf_a 0.06567411630

0.13767078601914476
muz_list -0.04672963412787093 mu_0_list 0.024725095196289876 sigma0_list 1.1164824175422434 sigmaz_list 0.04822990018988044 alpha_list -1.708764092533582 sf_a 0.06570257550203606 sf_b 1.4831448476025715 sf_c 4.450074153609954 sf_d 5.905621376172817
0.13767083071752084
muz_list -0.046817238321698554 mu_0_list 0.02473909379297371 sigma0_list 1.1167952675588275 sigmaz_list 0.048190703819521924 alpha_list -1.709101470886602 sf_a 0.0656680828697466 sf_b 1.4838766964323686 sf_c 4.449093193437758 sf_d 5.905595957649798
0.1376709334098788
muz_list -0.04679046385908808 mu_0_list 0.024737008478211153 sigma0_list 1.1166414716852504 sigmaz_list 0.048200878367879646 alpha_list -1.7085989098008394 sf_a 0.06568043429134457 sf_b 1.4836502335733217 sf_c 4.449342065262268 sf_d 5.905626726089958
0.13767084549177291
muz_list -0.04670786066744673 mu_0_list 0.024730143508703277 sigma0_list 1.1161582042076748 sigmaz_list 0.04821972923010105 alpha_list -1.7067085482051012 sf_a 0.0656910945

0.1376707357414055
muz_list -0.04678394389925464 mu_0_list 0.024733538539455324 sigma0_list 1.1165945369167383 sigmaz_list 0.0482002091257159 alpha_list -1.7085558238486573 sf_a 0.06569741968878814 sf_b 1.483326295968895 sf_c 4.449604254350904 sf_d 5.905331830971268
0.13767074515451475
muz_list -0.046759646044917384 mu_0_list 0.024729862555202476 sigma0_list 1.1165627903280373 sigmaz_list 0.04820896019086425 alpha_list -1.7086675848435307 sf_a 0.06571227209325803 sf_b 1.4830241286638148 sf_c 4.450110301798446 sf_d 5.905958691186433
0.13767072322010537
muz_list -0.04675935518642792 mu_0_list 0.024728416242107386 sigma0_list 1.1165422311911817 sigmaz_list 0.0482140360046758 alpha_list -1.708845601865833 sf_a 0.06571859970444849 sf_b 1.4828881413062347 sf_c 4.450369092682301 sf_d 5.906248159230998
0.13767077529782976
muz_list -0.04676278600783067 mu_0_list 0.024732484461896662 sigma0_list 1.1165365995466359 sigmaz_list 0.048200590362944065 alpha_list -1.7083656614530924 sf_a 0.06568909323

0.1376707076514714
muz_list -0.04677324372716907 mu_0_list 0.024731470072637138 sigma0_list 1.11660841246225 sigmaz_list 0.048198626208545176 alpha_list -1.7086342444003688 sf_a 0.06570437566383619 sf_b 1.4831671545914384 sf_c 4.449887590803816 sf_d 5.905831584663576
0.13767069610775004
muz_list -0.04676853696019136 mu_0_list 0.024729241123319402 sigma0_list 1.1165966827277691 sigmaz_list 0.04819900958942641 alpha_list -1.7086720256946746 sf_a 0.06570663705601652 sf_b 1.4830734738223936 sf_c 4.450064075796136 sf_d 5.906083507627432
0.13767069895514214
muz_list -0.04676304715985215 mu_0_list 0.02473033345716546 sigma0_list 1.116584805568611 sigmaz_list 0.04819783450401029 alpha_list -1.7085452275544553 sf_a 0.0657035962962674 sf_b 1.4831100021456756 sf_c 4.450075912438663 sf_d 5.906114473859928
0.13767070297854742
muz_list -0.04676633929480763 mu_0_list 0.024730347200237816 sigma0_list 1.116605165913524 sigmaz_list 0.048197976477529395 alpha_list -1.70862859767293 sf_a 0.065705583600558

0.13767068809339786
muz_list -0.04677542143140888 mu_0_list 0.024731165988526045 sigma0_list 1.1166225172441602 sigmaz_list 0.04819029579467814 alpha_list -1.708535248061604 sf_a 0.06570588114082293 sf_b 1.483169554638411 sf_c 4.4498045820361884 sf_d 5.9057616903224694
0.13767068937551064
muz_list -0.04677759314999425 mu_0_list 0.024730781030208537 sigma0_list 1.1166793685931466 sigmaz_list 0.04819037851780615 alpha_list -1.7087435253847247 sf_a 0.06570900065982896 sf_b 1.4831144730795585 sf_c 4.449897403091148 sf_d 5.905835462783209
0.1376706893049705
muz_list -0.0467745560397808 mu_0_list 0.02473107672451726 sigma0_list 1.1166421848450132 sigmaz_list 0.04818965791461666 alpha_list -1.7085977138378257 sf_a 0.06570398700262471 sf_b 1.4831799640975685 sf_c 4.449849901436505 sf_d 5.905876393033558
0.1376706890346887
muz_list -0.046771002298562916 mu_0_list 0.02473015152037085 sigma0_list 1.116645854172736 sigmaz_list 0.048192945823255534 alpha_list -1.7086859470816198 sf_a 0.065708216719

In [7]:
print(res.success, res.message, res.nit)
muz_best, mu0_best, sigma0_best, sigmaz_best, alpha_best = res.x[0], res.x[1], res.x[2], res.x[3],res.x[4]
sf_a_best, sf_b_best, sf_c_best, sf_d_best    = res.x[5], res.x[6], res.x[7], res.x[8] 

print('\nBEST FITTING PARAMETERS:')
print('muz =%s, mu0 =%s, sigma_0 =%s, sigma_z =%s, alpha=%s'% (muz_best, mu0_best, sigma0_best, sigmaz_best, alpha_best) )
print('sf_a =%s, sf_b =%s, sf_c =%s, sf_d =%s'% (sf_a_best, sf_b_best, sf_c_best, sf_d_best) )

if res.success:
    np.savetxt(paths.data / 'best_fit_parameters.txt', (mu0_best, muz_best, sigma0_best, sigmaz_best, alpha_best,sf_a_best, sf_b_best, sf_c_best, sf_d_best),fmt="%s")


True Optimization terminated successfully. 1717

BEST FITTING PARAMETERS:
muz =-0.04677378362710756, mu0 =0.024730821484976583, sigma_0 =1.1166378490971502, sigma_z =0.048190741547454244, alpha=-1.7086079887680787
sf_a =0.06570561539662137, sf_b =1.4831498367401643, sf_c =4.44988154529773, sf_d =5.905882609485851
